In [3]:
import pandas as pd
import chromadb
import string
import ast
import os

from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
from sentence_transformers import SentenceTransformer
from transformers import T5Tokenizer, T5ForConditionalGeneration

import warnings
warnings.filterwarnings("ignore")

### 1. Análisis exploratorio inicial

In [5]:
dn = pd.read_csv('../data/raw/Daniel_Noboa.csv')

In [6]:
dn

,Id,Candidato,Temas tratados,Descripción,Entrevista
0,DNE001,Daniel Noboa,"Formación y trayectoria, Seguridad ciudadana, ...",La entrevista con el presidente Daniel Noboa a...,Hola a todos Bienvenido a diálogo electoral ug...
1,DNE002,Daniel Noboa,Seguridad y Justicia; Desaparición de los cuat...,La entrevista con el presidente Daniel Noboa e...,la unidad nacional en pro de la paz y la segur...
2,DNE003,Daniel Noboa,Gobierno y Política; La democracia y sus desaf...,El conversatorio abordó temas clave como democ...,Aló Buenos días con todos por favor tomen asie...
3,DNP004,Daniel Noboa,Reducción de la pobreza y la desigualdad; Acce...,El documento presenta un plan integral para el...,"eguridad, desarrollo y bienestar para todos lo..."
4,DNP005,Daniel Noboa,Desarrollo Económico Sostenible; Transformació...,El documento presenta un plan integral para el...,Desarrollo económico sostenible para mejorar l...
5,DNP006,Daniel Noboa,"Medio Ambiente, Energía y Conectividad; Conser...",El documento presenta un enfoque integral para...,"Un Ecuador conectado, preparado para el futuro..."
6,DNP007,Daniel Noboa,Institucionalidad pública; AccÇeso a la inform...,El documento presenta un plan para la **transf...,Un nuevo Estado para un nuevo Ecuador: Eficien...


### 2. Preprocesar todo el corpus contenido en 'raw'

Tenemos dos tareas aquí:

1. Necesitamos solo archivos .csv, pero tenemos también algunos archivos en formato .xlsx, por lo que debemos primero convertir dichos archivos a .csv para tener un formato universal para nuestro corpus en 'raw'.
2. Preprocesar todo el corpus contenido en la carpeta 'raw', y dichos archivos preprocesados guardarlos en otra carpeta 'processed'.

In [ ]:
# Código para encontrar archivos .xlsx y guardarlos como .csv
raw_files = os.scandir('../data/raw')

for file in raw_files:
    if file.name.endswith('.xlsx'):
        df_xlsx = pd.read_excel(f'../data/raw/{file.name}')
        df_xlsx.to_csv(f'../data/raw/{file.name.split(".xlsx")[0]}.csv', index=False)

In [3]:
# Inicializar el stemmer en español
stemmer = SnowballStemmer("spanish")

# Cargar stopwords en español
stop_words = set(stopwords.words('spanish'))

# Definir caracteres de puntuación adicionales en español
puntuaciones = string.punctuation + "¿¡"

# Función de preprocesamiento mejorada
def preprocesar_texto(texto):
    # Convertir a minúsculas
    texto = texto.lower()

    # Eliminar puntuaciones y números
    texto = "".join([char for char in texto if char not in puntuaciones and not char.isdigit()])

    # Tokenizar
    tokens = word_tokenize(texto)

    # Eliminar stopwords y aplicar stemming
    tokens_procesados = [stemmer.stem(t) for t in tokens if t not in stop_words]

    return " ".join(tokens_procesados)

In [64]:
# Código para preprocesar todos los archivos .csv de la carpeta 'raw'
csv_files = os.scandir('../data/raw')

for file in csv_files:
    if file.name.endswith('.csv'):
        print(f'Iterando sobre {file.name}')
        df = pd.read_csv(f'../data/raw/{file.name}')
        df['temas_tratados'] = df['temas_tratados'].apply(preprocesar_texto)
        df['descripcion'] = df['descripcion'].apply(preprocesar_texto)
        df['entrevista'] = df['entrevista'].apply(preprocesar_texto)
        df.to_csv(f'../data/processed/{file.name}_processed.csv', index=False)

Iterando sobre Daniel_Noboa.csv
Iterando sobre entrevistas_carlos_rabascall_combined_final.csv
Iterando sobre Entrevistas_JimmyJairala.csv
Iterando sobre Entrevistas_JorgeEscala.csv
Iterando sobre entrevistas_pedro_granja_combined_final.csv
Iterando sobre Entrevista_Andrea_Gonzalez.csv
Iterando sobre Entrevista_LuisaGonzales_JuanCueva.csv
Iterando sobre FrancescoTabacchi.csv
Iterando sobre HenryCucalon.csv
Iterando sobre IvanSaquicela.csv
Iterando sobre Tilleria-Iza-Kronfle.csv
Iterando sobre victor_araus.csv
Iterando sobre Wilson_Gomez.csv


### 3. Unificación del corpus

Tenemos dos carpetas que contienen el corpus:

1. raw: Contiene los archivos sin procesar del corpus
2. processed: Contiene los archivos preprocesados del corpus

Debido a que se ha utilizado lematización al preprocesar el texto, este tiene la característica de recortar palabras, por lo que al recuperar los documentos que queremos no podemos mostrar eso al usuario, es por esto que se hará lo siguiente:

1. Concatenar todos los archivos de cada carpeta respectivamente, el resultado será dos archivos (raw_corpus.csv y processed_corpus.csv)
2. Se realizará un join entre estos dos archivos utilizando la columna 'ID' para esta unión.

In [65]:
base_dir = r'../data/raw'
processed_files = os.scandir(base_dir)
unified_raw_df = pd.DataFrame()

for file in processed_files:
    if file.name.endswith('.csv'):
        df = pd.read_csv(rf'{base_dir}/{file.name}')
        unified_raw_df = pd.concat([df, unified_raw_df])
    
del df

In [66]:
base_dir = r'../data/processed'
processed_files = os.scandir(base_dir)
unified_processed_df = pd.DataFrame()

for file in processed_files:
    df = pd.read_csv(rf'{base_dir}/{file.name}')
    unified_processed_df = pd.concat([df, unified_processed_df])

In [68]:
unified_raw_df['id'] = unified_raw_df['id'].astype(str)
unified_processed_df['id'] = unified_processed_df['id'].astype(str)

In [71]:
unified_global_df = pd.merge(unified_raw_df, unified_processed_df, on='id', suffixes=('_raw', '_pre'))

In [72]:
unified_global_df.drop(columns = ['descripcion_pre', 'temas_tratados_pre', 'candidato_pre'], inplace=True)
unified_global_df.head(10)

,id,candidato_raw,temas_tratados_raw,descripcion_raw,entrevista_raw,entrevista_pre
0,WGE001,Wilson Gomez,Conformación del binomio presidencial; Relació...,El documento es una entrevista en la que un ca...,Yo vengo trabajando para muchos presidentes he...,veng trabaj president oportun serv voluntari e...
1,WGE002,Wilson Gomez,Crisis del país; Propuestas de rescate y trans...,El documento expone la visión y propuestas del...,Buenos días nos encontramos con Wilson Gómez c...,buen dias encontr wilson gomez candidat presid...
2,WGE003,Wilson Gomez,La necesidad de un cambio en el modelo educati...,El documento es una entrevista con el candidat...,un gran abrazo para la inmensa audiencia de ra...,gran abraz inmens audienci radi moren ecuador ...
3,WGP004,Wilson Gomez,"El rol de la agricultura, la pesca y la indust...",El documento analiza el sector agropecuario y ...,Parte 1: Diagnóstico de la Situación Actual La...,part diagnost situacion actual agricultur pesc...
4,WGP005,Wilson Gomez,Extracción de petróleo; Historia y auge de la ...,El documento analiza tres sectores clave en Ec...,Parte 1: Extracción de petróleo en Ecuador Dia...,part extraccion petrole ecuador diagnost situa...
5,WGP006,Wilson Gomez,Definición y módulos de FinTech; Criptomonedas...,"El documento analiza el sector FinTech, su imp...",Parte 1: ¿Qué es FinTech y cuáles son sus módu...,part fintech cual modul principal termin finte...
6,WGP007,Wilson Gomez,Economía agraria; Industrialización y auge pet...,El texto describe la evolución económica de Ec...,EXTRACCIÓN DE ECONOMÍA & PRODUCTIVIDAD Histori...,extraccion econom product histori econom ecuad...
7,WGP008,Wilson Gomez,Ecoturismo en Ecuador; Economía y Productivida...,El documento propone un plan de desarrollo par...,Estrategia para el Desarrollo del Ecoturismo e...,estrategi desarroll ecotur ecuador problem afe...
8,VA1,Victor Araus,"Retiro de la visa de Víctor Araus, conspiració...",La primera entrevista se centró en sus propues...,Buenos días gracias por acompañarnos Gracias a...,buen dias graci acompañ graci usted invit salu...
9,VA2,Victor Araus,"Seguridad y combate a la delincuencia, elimina...","En esta entrevista, Víctor Araus presentó su v...",de la mañana con 55 minutos le vamos a dar la ...,mañan minut vam dar bienven general victor ara...


In [73]:
unified_raw_df.to_csv(rf'../data/raw/raw_corpus.csv', index=False)
unified_processed_df.to_csv(rf'../data/processed/processed_corpus.csv', index=False)
unified_global_df.to_csv(rf'../data/embeddings/unified_corpus.csv', index=False)

### 4. Generación de embeddings

Una vez tenemos todo el corpus unificado en un único archivo, tenemos que calcular los embeddings para el contenido de cada entrevista, esto lo haremos utilizando la libreria SentenceTransformer.

In [4]:
unified_corpus = pd.read_csv('../data/embeddings/unified_corpus.csv')

In [5]:
# Modelo con el que vamos a generar embeddings
modelo = SentenceTransformer("all-MiniLM-L6-v2")

# Generación de los embeddings a partir de corpus preprocesado
unified_corpus['embedding_entrevista'] = unified_corpus['entrevista_pre'].apply(lambda x: modelo.encode(x).tolist())

In [77]:
unified_corpus['embedding_entrevista']

0     [-0.014551389962434769, -0.0037382026202976704...
1     [0.007717538625001907, -0.04105502739548683, -...
2     [-0.007631767075508833, -0.03830636292695999, ...
3     [0.01490846835076809, -0.08086168766021729, -0...
4     [0.013256512582302094, -0.06549493223428726, -...
5     [-0.008189497515559196, -0.02202538400888443, ...
6     [0.020869851112365723, -0.09945572167634964, 0...
7     [0.06572176516056061, -0.0940171629190445, -0....
8     [-0.02826605550944805, 0.026913078501820564, -...
9     [-0.041109848767519, -0.02618183009326458, 0.0...
10    [-0.03815913945436478, 0.0021859726402908564, ...
11    [0.02806268073618412, -0.021873988211154938, -...
12    [0.0787014365196228, -0.07890287786722183, -0....
13    [0.04036758840084076, -0.09257502853870392, -0...
14    [0.012605945579707623, 0.02397148311138153, -0...
15    [0.0333082340657711, -0.0050742304883897305, -...
16    [0.006792896892875433, -0.019541403278708458, ...
17    [-0.0605597123503685, 0.013441740535199642

In [78]:
unified_corpus.to_csv(rf'../data/embeddings/unified_corpus_embeddings.csv', index=False)

### 5. Base de datos vectorial con ChromaDB

Una vez tenemos un único archivo que contiene todo el corpus junto a los embeddings del contenido de cada entrevista, vamos a construir una base de datos vectorial con ChromaDB para la realización de consultas.

La estructura a manejar será la siguiente:
- ID: El identificador alfanumérico de cada entrevista
- embeddings: El vector representativo del contenido de la entrevista
- metadatos: El nombre del candidato, tema tratado y una descripción de la entrevista

In [4]:
# Creación de la base de datos vectorial
chroma_client = chromadb.PersistentClient(path="../db/")
collection = chroma_client.get_or_create_collection("entrevistas")

In [80]:
embeddings_df = pd.read_csv('../data/embeddings/unified_corpus_embeddings.csv')

In [81]:
embeddings_df['embedding_entrevista']

0     [-0.014551389962434769, -0.0037382026202976704...
1     [0.007717538625001907, -0.04105502739548683, -...
2     [-0.007631767075508833, -0.03830636292695999, ...
3     [0.01490846835076809, -0.08086168766021729, -0...
4     [0.013256512582302094, -0.06549493223428726, -...
5     [-0.008189497515559196, -0.02202538400888443, ...
6     [0.020869851112365723, -0.09945572167634964, 0...
7     [0.06572176516056061, -0.0940171629190445, -0....
8     [-0.02826605550944805, 0.026913078501820564, -...
9     [-0.041109848767519, -0.02618183009326458, 0.0...
10    [-0.03815913945436478, 0.0021859726402908564, ...
11    [0.02806268073618412, -0.021873988211154938, -...
12    [0.0787014365196228, -0.07890287786722183, -0....
13    [0.04036758840084076, -0.09257502853870392, -0...
14    [0.012605945579707623, 0.02397148311138153, -0...
15    [0.0333082340657711, -0.0050742304883897305, -...
16    [0.006792896892875433, -0.019541403278708458, ...
17    [-0.0605597123503685, 0.013441740535199642

In [83]:
for index, row in embeddings_df.iterrows():
    collection.add(
        ids = [row['id']],
        documents = [row['entrevista_raw']],
        embeddings = [ast.literal_eval(row['embedding_entrevista'])],
        metadatas = [{
            'candidato': row['candidato_raw'],
            'temas_tratados': row['temas_tratados_raw'],
            'descripcion': row['descripcion_raw']
        }]
    )

### 6. Ejemplo de consultas

In [9]:
query = 'Vamos a reducir los asaltos en el país'

# Preprocesar la consulta
preproc_query = preprocesar_texto(query)

# Generar el embedding de la consulta
embedding_query = modelo.encode(preproc_query)

In [10]:
# Realizar la consulta en la base de datos
resultados = collection.query(query_embeddings=[embedding_query], n_results=3)  # Los 3 resultados más cercanos

# Mostrar los resultados
print("Resultados de la consulta:")
print()
for i in range(len(resultados['documents'][0])):
    print(f"Resultado {i+1}")
    print(f"ID: {resultados['ids'][0][i]}")
    print(f"Contenido: {resultados['documents'][0][i]}")
    print(f"Metadatos: {resultados['metadatas'][0][i]}")
    print("")

Resultados de la consulta:

Resultado 1
ID: FT3
Contenido:  ¿Qué es lo que es la historia del país? ¿Qué es el país del país del país? ¿Qué es el país del país del país del país del país? ¿Qué es el país del país del país del país del país? La misma pregunta me hace mi familia, mi esposa, mis hijos, mis padres, me hacen mis socios. Y este país ha sido muy generoso conmigo. Me he caído, me he levantado. Pero, afinitivamente, este país ha sido bueno conmigo. Víe el momento y la forma de retribuírselo. Creo en mi país, creo en el Ecuador. Porque ahora, si bien usted ha estado ligado a las partes ganaderas, productivas, a la parte del agro, al momento de ingresar a la vida política, ¿qué le...? ¿Cómo soles interés? No solo por ingresar, sino por mantenerse ahí. Porque después, eso pasó como gobernador, tuvo una candidatura de refectura también, ahora salta la presidencia. ¿A qué se debe este interés en seguir viendo también que es una dinámica muy distinta a la que a lo mejor estaba acostu

In [86]:
resultados["documents"][0]

[' ¿Qué es lo que es la historia del país? ¿Qué es el país del país del país? ¿Qué es el país del país del país del país del país? ¿Qué es el país del país del país del país del país? La misma pregunta me hace mi familia, mi esposa, mis hijos, mis padres, me hacen mis socios. Y este país ha sido muy generoso conmigo. Me he caído, me he levantado. Pero, afinitivamente, este país ha sido bueno conmigo. Víe el momento y la forma de retribuírselo. Creo en mi país, creo en el Ecuador. Porque ahora, si bien usted ha estado ligado a las partes ganaderas, productivas, a la parte del agro, al momento de ingresar a la vida política, ¿qué le...? ¿Cómo soles interés? No solo por ingresar, sino por mantenerse ahí. Porque después, eso pasó como gobernador, tuvo una candidatura de refectura también, ahora salta la presidencia. ¿A qué se debe este interés en seguir viendo también que es una dinámica muy distinta a la que a lo mejor estaba acostumbrada previamente? Quizás después de ser gobernador, me 

In [87]:
resultados['metadatas'][0]

[{'candidato': 'Francesco Tabacchi',
  'descripcion': 'Francesco Tabaqui, candidato presidencial por Creo, expone su visión para el país basada en seguridad, generación de empleo y apertura a la inversión privada en sectores estratégicos como energía, minería y petróleo. Propone una política de "mano dura, mano justa y mano inteligente", inspirada en el modelo de El Salvador, con la creación del Escuadrón de la Pacificación para combatir el crimen organizado. En economía, plantea incentivos fiscales para fomentar el empleo y la modernización del sector agropecuario con una "agricultura 4.0". Rechaza la idea de modificar la Constitución y en su lugar aboga por reformas rápidas para mejorar la seguridad y la economía. También sugiere concesionar la generación eléctrica e infraestructura vial, y se muestra abierto a revisar el sistema de subsidios. En política exterior, busca mantener relaciones pragmáticas con EE.UU., México y Venezuela, aunque sin reconocer a Nicolás Maduro. Destaca la 